## Homework 3

In this assignment, we will practice implementing value iteration in the [Frozen Lake environment](https://gymnasium.farama.org/environments/toy_text/frozen_lake/) from OpenAI Gym, and implementing REINFORCE policy gradient algorithm for [Cart-Pole v1](https://gymnasium.farama.org/environments/classic_control/cart_pole/) environment.

* You should run this on Google Colab




## Bellman Update and Value Iteration

In [ ]:
# IMPORTANT: Always run this cell before anything else to ensure that you are able to access the Frozen Lake environment
!pip install gymnasium==1.0.0
import gymnasium as gym
import argparse
import numpy as np
import time
from gymnasium.envs.registration import register

# De-register environments if there is a collision
env_dict = gym.envs.registration.registry.copy()
for env in env_dict:
    if "Deterministic-4x4-FrozenLake-v0" in env:
        del gym.envs.registration.registry[env]
    elif "Stochastic-4x4-FrozenLake-v0" in env:
        del gym.envs.registration.registry[env]


register(
    id="Deterministic-4x4-FrozenLake-v0",
    entry_point="gymnasium.envs.toy_text.frozen_lake:FrozenLakeEnv",
    kwargs={"map_name": "4x4", "is_slippery": False},
)

register(
    id="Stochastic-4x4-FrozenLake-v0",
    entry_point="gymnasium.envs.toy_text.frozen_lake:FrozenLakeEnv",
    kwargs={"map_name": "4x4", "is_slippery": True},
)





The parameters P, nS, nA, gamma are defined as follows:

	P: nested dictionary of a nested lists
		From gym.core.Environment
		For each pair of states in [1, nS] and actions in [1, nA], P[state][action] is a
		tuple of the form (probability, nextstate, reward, terminal) where
			- probability: float
				the probability of transitioning from "state" to "nextstate" with "action"
			- nextstate: int
				denotes the state we transition to (in range [0, nS - 1])
			- reward: int
				either 0 or 1, the reward for transitioning from "state" to
				"nextstate" with "action"
			- terminal: bool
			  True when "nextstate" is a terminal state (hole or goal), False otherwise
	nS: int
		number of states in the environment
	nA: int
		number of actions in the environment
	gamma: float
		Discount factor. Number in range [0, 1)

In [ ]:
env = gym.make('Stochastic-4x4-FrozenLake-v0')

env.unwrapped.nS = env.unwrapped.nrow * env.unwrapped.ncol
env.unwrapped.nA = 4

In [ ]:
current_state = 3 # modify between 0~15
current_action = 0 # modify between 0~3
print(env.unwrapped.P[current_state][current_action])

# sample output: [(1.0, 2, 0.0, False)], it transits to state 2 with
# probabilty 1, receives reward 0, not terminating.

# sample output: [(0.3333333333333333, 3, 0.0, False), (0.3333333333333333, 2,
# 0.0, False), (0.3333333333333333, 7, 0.0, True)]

Recall that if we repeatedly apply the Bellman Operator to the value function, it will converges to the optimal value function. First, we implement the Bellman Update function below.

In [ ]:
def Bellman_Update(P, nS, nA, value, gamma=0.9):
		"""
		Given the value function, apply Bellman Operator to the value function.

	Parameters
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	value: np.ndarray, shape = (nS, )
		The input value function

	Returns
	-------
	new_value: np.ndarray, shape = (nS, )
		the improved value function.
	greedy_policy: np.ndarray, shape = (nS, )
		the greedy policy from the improved value function.
	"""

		new_value = np.zeros(nS) # TODO
		greedy_policy = np.zeros(nS, dtype="int") # TODO

		### Q1: Bellman Update, 15 points ###
		############################
		# YOUR IMPLEMENTATION HERE #

		############################

		return new_value, greedy_policy


Next, we run a for loop to apply Bellman Updates repeatedly to an initial value.

In [ ]:
def value_iteration(P, nS, nA, gamma=0.9, tol=1e-3):
		"""
	Performs value iteration by repeatedly applying Bellman Updates.

	Parameters:
	----------
	P, nS, nA, gamma:
		defined at beginning of file
	tol: float
		Terminate value iteration when
			max |value_function(s) - prev_value_function(s)| < tol
	Returns:
	----------
	policy: np.ndarray[nS]
		the greedy policy derived from the final value function
	"""

		value_function = np.zeros(nS) # initial value function
		policy = np.zeros(nS, dtype=int) # TODO

		### Q2: Value Iteration, 5 points ###
		############################
		# YOUR IMPLEMENTATION HERE #

		############################
		return policy

We provide you with the following function to evaluate how good your policy is, by interfering with the environment!

In [ ]:
def evaluate(env, policy, max_steps=100):
    """
    This function does not need to be modified
    Watch your agent play!

    Parameters
    ----------
    env: gym.core.Environment
      Environment to play on. Must have nS, nA, and P as
      attributes.
    Policy: np.array of shape [env.nS]
      The action to take at a given state
  """

    episode_reward = 0
    ob, _ = env.reset()
    for t in range(max_steps):
        a = policy[ob]
        ob, rew, done, _, _ = env.step(a)
        episode_reward += rew
        if done:
            break
    if not done:
        print(
            "The agent didn't reach a terminal state in {} steps.".format(
                max_steps
            )
        )
    else:
        print("Episode reward: %f" % episode_reward)


In [ ]:
# Run the code below to implement value iteration on Frozen Lake!
# You may change the parameters in the functions below
np.set_printoptions(precision=3)

# Make gym environment
env = gym.make('Stochastic-4x4-FrozenLake-v0')

env.unwrapped.nS = env.unwrapped.nrow * env.unwrapped.ncol
env.unwrapped.nA = 4

print("\n" + "-" * 25 + "\nBeginning Value Iteration\n" + "-" * 25)

p_vi = value_iteration(env.unwrapped.P, env.unwrapped.nS, env.unwrapped.nA, gamma=0.9, tol=1e-3)
print('the learned policy:', p_vi)
evaluate(env, p_vi, 100)

## successful policy should have episode reward close to 1, and the agent should terminate within 100 steps.

## Policy Gradient: REINFORCE

In this problem, we try to implement the `REINFORCE` algorithm to learn the optimal policy on environment `CartPole-v1` of OpenAI-Gym. We will use a neural network to parametrize the policy, and then run policy gradient on it. We  provide you with the following evaluation function, to calculate the expected cumulative reward of your current policy through Monte Carlo. You need to (1) implement REINFORCE with a neural network, and (2) plot the expected reward againt the number of training epochs.

In [ ]:
def evaluate_neural(env, policy, max_steps=500, trials = 100):
    """
    This function does not need to be modified
    Renders policy once on environment. Watch your agent play!

    Parameters
    ----------
    env: gym.core.Environment
    Policy: torch.Distribution, trained with REINFORCE
    trials: int, number of trials for Monte Carlo, default = 100
    Returns
    -------
    cum_reward_mean: estimate of the expected cumulative reward
  """
    cum_reward = []
    for i in range(trials):
      episode_reward = 0
      ob,_ = env.reset()
      for t in range(max_steps):
          ob = torch.tensor(ob, dtype=torch.float32)
          a = Categorical(policy(ob)).sample().item()
          ob, rew, done, _, _ = env.step(a)
          episode_reward += rew
          if done:
              break
      cum_reward.append(episode_reward)
    cum_reward = np.array(cum_reward)
    return cum_reward.mean(), cum_reward.std()

Now you are ready to implement the algorithm! In the following we define a class `PolicyNetwork` to be 2-layer ReLU network. Recall that the (variance reduced) gradient estimate in `REINFORCE` can be defined by
$$ \sum_{h\geq 0} \nabla_\theta\log \pi_\theta(A_h|S_h) \cdot \sum_{t\geq h}\gamma^{t-h} r(S_t, A_t).  $$

 You have the freedom to modify the network architecture and the training procedure of the neural network, but make sure that it takes the state as input, and outputs a distrubution for the actions! Once it is implemented and trained, plot the cumulative reward with error bar.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

# Define the policy network
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PolicyNetwork, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.softmax(x, dim=-1)

In [ ]:
# Initialize environment and optimizer. Feel free to change!
env = gym.make('CartPole-v1')
input_size = 4
output_size = env.action_space.n
hidden_size = 64


policy_net = PolicyNetwork(input_size, hidden_size, output_size)
optimizer = optim.Adam(policy_net.parameters(), lr=0.001)
# Training hyperparameters. Feel free to change!
num_episodes = 3000
gamma = 0.99

In [ ]:
## statistics
episode_ids = []
avg_rewards = []
std_rewards = []


# Training loop
for episode in range(num_episodes):
    episode_rewards = []
    log_probs = []

    state,_ = env.reset()
    done = False

    while not done:
      # Sample action from the policy network. For simplicity we only use single trajectory
      # to update. But feel free to change this into batch learning!
      state_tensor = torch.from_numpy(state).float().unsqueeze(0)
      action_probs = policy_net(state_tensor)
      action_dist = Categorical(action_probs)
      action = action_dist.sample()
      log_prob = action_dist.log_prob(action)
      log_probs.append(log_prob)

      # Take action and observe next state and reward
      next_state, reward, done, _, _ = env.step(action.item())
      episode_rewards.append(reward)
      state = next_state


    # Now use the episode_rewards list, the log_prob list to construct a loss function, on which you can
    # backward propagate and optimize with optimizer.

    # First, compute the discounted cumulative reward for every step in the trajectory.
    # G_t = r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + ...
    # Hint: compute this in backward direction.
    discounted_rewards = []

    ### Q3: discounted rewards, 15 points ###
    ############################
		# YOUR IMPLEMENTATION HERE #

    ############################

    discounted_rewards = torch.tensor(discounted_rewards, dtype=torch.float32)
    optimizer.zero_grad()


    # Compute policy loss and update policy network
    ## note that `log_probs` contains differentiable torch.tensor computed through the policy network
    ## construct a loss using `log_probs` via REINFORCE, and call loss.backward()

    ### Q4: REINFORCE, 15 points ###
    ############################
		# YOUR IMPLEMENTATION HERE #

    ############################

    policy_loss.backward()
    optimizer.step()

    # Record the cumulative reward and its deviation once every 100 episodes
    if (episode + 1) % 100 == 0:
      avg_reward, std_reward = evaluate_neural(env, policy_net)
      print(f'Episode [{episode + 1}/{num_episodes}], Cumulative Reward: {avg_reward}')
      episode_ids.append(episode + 1)
      avg_rewards.append(avg_reward)
      std_rewards.append(std_reward)

In [ ]:
from matplotlib import pyplot as plt
# Plot the curve of cumulative reward v.s. number of episodes, with error bar

plt.errorbar(episode_ids, avg_rewards, std_rewards, label='REINFORCE')
plt.xlabel("Num Episode")
plt.ylabel("Cumulative Rewards")
plt.legend()

## optimal policy should reach near 500 of cumulative rewards.